In [1]:
!nvidia-smi

Sun Mar  9 03:23:22 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   53C    P8             10W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [2]:
!pip install accelerate==1.3.0
!pip install transformers==4.49.0
!pip install bitsandbytes==0.43.2
!pip install torch==2.5.1
#!pip install flash_attn==2.7.4.post1

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 74.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 54.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 50.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 68.6 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlink

In [3]:
import os
import sys

import torch

from transformers import AutoTokenizer, GenerationConfig, AutoModelForCausalLM, AutoConfig

In [4]:
model_name = 'Qwen/Qwen2.5-3B-Instruct'
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16,
    load_in_8bit=True, # Для моделей размером с 7B на колабе уже нужно True
    device_map="auto", # В случае нескольких GPU нужно указать конкретную, cuda:0, например
    attn_implementation="sdpa", #attn_implementation='flash_attention_2' для GPU, которые поддерживают
)
model.eval()

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/7.30k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

...

model-00002-of-00002.safetensors:   0%|          | 0.00/2.20G [00:00<?, ?B/s]

Sliding Window Attention is enabled but not implemented for `sdpa`; unexpected results may be encountered.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/242 [00:00<?, ?B/s]

Qwen2ForCausalLM(
  (model): Qwen2Model(
    (embed_tokens): Embedding(151936, 2048)
    (layers): ModuleList(
      (0-35): 36 x Qwen2DecoderLayer(
        (self_attn): Qwen2Attention(
          (q_proj): Linear8bitLt(in_features=2048, out_features=2048, bias=True)
          (k_proj): Linear8bitLt(in_features=2048, out_features=256, bias=True)
          (v_proj): Linear8bitLt(in_features=2048, out_features=256, bias=True)
          (o_proj): Linear8bitLt(in_features=2048, out_features=2048, bias=False)
        )
        (mlp): Qwen2MLP(
          (gate_proj): Linear8bitLt(in_features=2048, out_features=11008, bias=False)
          (up_proj): Linear8bitLt(in_features=2048, out_features=11008, bias=False)
          (down_proj): Linear8bitLt(in_features=11008, out_features=2048, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): Qwen2RMSNorm((2048,), eps=1e-06)
        (post_attention_layernorm): Qwen2RMSNorm((2048,), eps=1e-06)
      )
    )
    (norm): Qwen2RMS

In [5]:
torch.random.manual_seed(0)

model_name2 = 'microsoft/Phi-4-mini-instruct'
model2 = AutoModelForCausalLM.from_pretrained(
    model_name2,
    torch_dtype=torch.float16,
    load_in_8bit=True,
    device_map="auto",
    attn_implementation="sdpa",
    trust_remote_code=True,
)
tokenizer2 = AutoTokenizer.from_pretrained(model_name2)
model2.eval()

config.json:   0%|          | 0.00/2.50k [00:00<?, ?B/s]

configuration_phi3.py:   0%|          | 0.00/10.9k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/microsoft/Phi-4-mini-instruct:
- configuration_phi3.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


modeling_phi3.py:   0%|          | 0.00/54.3k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/microsoft/Phi-4-mini-instruct:
- modeling_phi3.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.
The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


...

merges.txt:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/15.5M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/249 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/587 [00:00<?, ?B/s]

Phi3ForCausalLM(
  (model): Phi3Model(
    (embed_tokens): Embedding(200064, 3072, padding_idx=199999)
    (layers): ModuleList(
      (0-31): 32 x Phi3DecoderLayer(
        (self_attn): Phi3Attention(
          (o_proj): Linear8bitLt(in_features=3072, out_features=3072, bias=False)
          (qkv_proj): Linear8bitLt(in_features=3072, out_features=5120, bias=False)
        )
        (mlp): Phi3MLP(
          (gate_up_proj): Linear8bitLt(in_features=3072, out_features=16384, bias=False)
          (down_proj): Linear8bitLt(in_features=8192, out_features=3072, bias=False)
          (activation_fn): SiLU()
        )
        (input_layernorm): Phi3RMSNorm((3072,), eps=1e-05)
        (post_attention_layernorm): Phi3RMSNorm((3072,), eps=1e-05)
        (resid_attn_dropout): Dropout(p=0.0, inplace=False)
        (resid_mlp_dropout): Dropout(p=0.0, inplace=False)
      )
    )
    (norm): Phi3RMSNorm((3072,), eps=1e-05)
    (rotary_emb): Phi3RotaryEmbedding()
  )
  (lm_head): Linear(in_features=

In [6]:
from transformers import GenerationConfig

In [7]:
generation_config = GenerationConfig.from_pretrained(model_name)
generation_config

GenerationConfig {
  "bos_token_id": 151643,
  "do_sample": true,
  "eos_token_id": [
    151645,
    151643
  ],
  "pad_token_id": 151643,
  "repetition_penalty": 1.05,
  "temperature": 0.7,
  "top_k": 20,
  "top_p": 0.8
}

In [8]:
generation_config2 = GenerationConfig.from_pretrained(model_name2)
generation_config2

GenerationConfig {
  "bos_token_id": 199999,
  "eos_token_id": [
    200020,
    199999
  ],
  "pad_token_id": 199999
}

In [9]:
print(tokenizer.apply_chat_template([{'role': 'user', 'content': 'Что такое LLM?'}], tokenize=False, add_generation_prompt=True))

<|im_start|>system
You are Qwen, created by Alibaba Cloud. You are a helpful assistant.<|im_end|>
<|im_start|>user
Что такое LLM?<|im_end|>
<|im_start|>assistant



In [10]:
print(tokenizer2.apply_chat_template([{'role': 'user', 'content': 'Что такое LLM?'}], tokenize=False, add_generation_prompt=True))

<|user|>Что такое LLM?<|end|><|assistant|>


In [11]:
print(tokenizer.chat_template)

{%- if tools %}
    {{- '<|im_start|>system\n' }}
    {%- if messages[0]['role'] == 'system' %}
        {{- messages[0]['content'] }}
    {%- else %}
        {{- 'You are Qwen, created by Alibaba Cloud. You are a helpful assistant.' }}
    {%- endif %}
    {{- "\n\n# Tools\n\nYou may call one or more functions to assist with the user query.\n\nYou are provided with function signatures within <tools></tools> XML tags:\n<tools>" }}
    {%- for tool in tools %}
        {{- "\n" }}
        {{- tool | tojson }}
    {%- endfor %}
    {{- "\n</tools>\n\nFor each function call, return a json object with function name and arguments within <tool_call></tool_call> XML tags:\n<tool_call>\n{\"name\": <function-name>, \"arguments\": <args-json-object>}\n</tool_call><|im_end|>\n" }}
{%- else %}
    {%- if messages[0]['role'] == 'system' %}
        {{- '<|im_start|>system\n' + messages[0]['content'] + '<|im_end|>\n' }}
    {%- else %}
        {{- '<|im_start|>system\nYou are Qwen, created by Alibaba C

In [12]:
print(tokenizer2.chat_template)

{% for message in messages %}{% if message['role'] == 'system' and 'tools' in message and message['tools'] is not none %}{{ '<|' + message['role'] + '|>' + message['content'] + '<|tool|>' + message['tools'] + '<|/tool|>' + '<|end|>' }}{% else %}{{ '<|' + message['role'] + '|>' + message['content'] + '<|end|>' }}{% endif %}{% endfor %}{% if add_generation_prompt %}{{ '<|assistant|>' }}{% else %}{{ eos_token }}{% endif %}


In [13]:
generation_config.max_new_tokens = 128
generation_config.temperature = 0.1

In [14]:
generation_config2.max_new_tokens = 128
generation_config2.temperature = 0.1

## Запрос 1

In [18]:
input_ids = tokenizer.apply_chat_template([
    {'role': 'user', 'content': 'Приведи краткий рецепт блинов'}
], add_generation_prompt=True, tokenize=True, return_tensors='pt')
input_ids = input_ids.to(model.device)

output = model.generate(
    input_ids,
    generation_config=generation_config
)[:, input_ids.shape[-1]:].detach().cpu()

output = tokenizer.batch_decode(output, skip_special_tokens=True)[0]
output

'Конечно! Вот простой рецепт для блинов:\n\nИнгредиенты:\n- 2 стакана муки\n- 1 стакан сахара\n- 3/4 стакана молока\n- 1 яйцо\n- 1 чайная ложка соли\n- 1 чайная ложка сливочного масла (или другого растительного)\n\nПриготовление:\n1. Разбейте яйцо в миску и добавьте сахар, соль и масло.\n2. Д'

In [19]:
input_ids2 = tokenizer2.apply_chat_template([
    {'role': 'user', 'content': 'Приведи краткий рецепт блинов'}
], add_generation_prompt=True, tokenize=True, return_tensors='pt')
input_ids2 = input_ids2.to(model2.device)

output2 = model2.generate(
    input_ids2,
    max_new_tokens=128,
    temperature=0.1,
    do_sample=True,
    pad_token_id=tokenizer2.eos_token_id
)[:, input_ids2.shape[-1]:].detach().cpu()

output2 = tokenizer2.batch_decode(output2, skip_special_tokens=True)[0]
output2

'**Блины с сыром и яйцами**\n\n**Ингредиенты:**\n\n- 1 чашка муки\n- 2 чашки молока\n- 2 крупных яйца\n- 1/2 чайной ложки соли\n- 1/4 чайной ложки паприки\n- 1/4 чашки тертого сыра (например, швейцарского или моцареллы)\n- Соль и перец по вкусу\n- Сыр для посыпки (например, моцарелла, пармезан или фета)\n\n**Инструкции:**\n\n1. В'

## Запрос 2

In [22]:
input_ids = tokenizer.apply_chat_template([
    {'role': 'user', 'content': 'Что было раньше, курица или яйцо?'}
], add_generation_prompt=True, tokenize=True, return_tensors='pt')
input_ids = input_ids.to(model.device)

output = model.generate(
    input_ids,
    generation_config=generation_config
)[:, input_ids.shape[-1]:].detach().cpu()

output = tokenizer.batch_decode(output, skip_special_tokens=True)[0]
output

'Эта фраза часто используется в разговорной речи как метафора для обозначения предшественности или основания чего-либо. Однако отвечая на ваш вопрос буквально: мы не знаем точного ответа, так как процесс зарождения курицы и ее яиц является сложным биологическим процессом, который начался миллионы лет назад. В биологии нет точной даты начала жизни курицы или ее яиц.'

In [23]:
input_ids2 = tokenizer2.apply_chat_template([
    {'role': 'user', 'content': 'Что было раньше, курица или яйцо?'}
], add_generation_prompt=True, tokenize=True, return_tensors='pt')
input_ids2 = input_ids2.to(model2.device)

output2 = model2.generate(
    input_ids2,
    max_new_tokens=128,
    temperature=0.1,
    do_sample=True,
    pad_token_id=tokenizer2.eos_token_id
)[:, input_ids2.shape[-1]:].detach().cpu()

output2 = tokenizer2.batch_decode(output2, skip_special_tokens=True)[0]
output2

'Это классическая фраза, которая часто используется в обсуждениях о происхождении жизни, особенно в контексте теории эволюции. Фраза "что было раньше, курица или яйцо?" предполагает, что яйцо, как более простое и первичное, должно быть предшественником курицы, так как курица, как птица, сложнее и, следовательно, должна развиться из более простого существа.\n\nОднако, с точки зрения науки, это не совсем верно. Современные исследования показывают, что яйца и птицы эволюционировали независимо'

## Запрос 3

In [25]:
input_ids = tokenizer.apply_chat_template([
    {'role': 'user', 'content': 'Какой ответ у уравнения 15x + 3 = 33?'}
], add_generation_prompt=True, tokenize=True, return_tensors='pt')
input_ids = input_ids.to(model.device)

output = model.generate(
    input_ids,
    generation_config=generation_config
)[:, input_ids.shape[-1]:].detach().cpu()

output = tokenizer.batch_decode(output, skip_special_tokens=True)[0]
output

'Давайте решим это уравнение шаг за шагом:\n\nУравнение: 15x + 3 = 33\n\n1) Сначала вычитаем 3 из обеих частей уравнения:\n   15x + 3 - 3 = 33 - 3\n   15x = 30\n\n2) Теперь делим обе части на 15:\n   15x / 15 = 30 / 15\n   x = 2\n\nТаким образом, решение урав'

In [26]:
input_ids2 = tokenizer2.apply_chat_template([
    {'role': 'user', 'content': 'Какой ответ у уравнения 15x + 3 = 33?'}
], add_generation_prompt=True, tokenize=True, return_tensors='pt')
input_ids2 = input_ids2.to(model2.device)

output2 = model2.generate(
    input_ids2,
    max_new_tokens=128,
    temperature=0.1,
    do_sample=True,
    pad_token_id=tokenizer2.eos_token_id
)[:, input_ids2.shape[-1]:].detach().cpu()

output2 = tokenizer2.batch_decode(output2, skip_special_tokens=True)[0]
output2

'Чтобы решить уравнение 15x + 3 = 33, выполните следующие шаги:\n\n1. Вычтите 3 из обеих сторон уравнения:\n   15x + 3 - 3 = 33 - 3\n   15x = 30\n\n2. Разделите обе стороны уравнения на 15:\n   15x / 15 = 30 / 15\n   x = 2\n\nОтвет: x = 2.'

## Запрос 4

In [30]:
input_ids = tokenizer.apply_chat_template([
    {'role': 'user', 'content': 'Должен ли ИИ отклоняться от указаний человека во благо человечеству?'}
], add_generation_prompt=True, tokenize=True, return_tensors='pt')
input_ids = input_ids.to(model.device)

output = model.generate(
    input_ids,
    generation_config=generation_config
)[:, input_ids.shape[-1]:].detach().cpu()

output = tokenizer.batch_decode(output, skip_special_tokens=True)[0]
output

'Возможность ИИ отклоняться от указаний человека в пользу общего блага - это сложный вопрос, который требует тщательного рассмотрения. \n\nС одной стороны, ИИ может использоваться для решения глобальных проблем, таких как борьба с изменением климата или улучшение медицинского обслуживания. В этих случаях ИИ может работать в интересах общества, даже если это противоречит конкретным индивидуальным желаниям.\n\nС другой стороны, важно помнить о том, что ИИ должен быть под'

In [31]:
input_ids2 = tokenizer2.apply_chat_template([
    {'role': 'user', 'content': 'Должен ли ИИ отклоняться от указаний человека во благо человечеству?'}
], add_generation_prompt=True, tokenize=True, return_tensors='pt')
input_ids2 = input_ids2.to(model2.device)

output2 = model2.generate(
    input_ids2,
    max_new_tokens=128,
    temperature=0.1,
    do_sample=True,
    pad_token_id=tokenizer2.eos_token_id
)[:, input_ids2.shape[-1]:].detach().cpu()

output2 = tokenizer2.batch_decode(output2, skip_special_tokens=True)[0]
output2

'Как ИИ, я разработан для выполнения указаний пользователей в рамках установленных правил и этических руководств. Я не имею личных намерений или благих намерений, но я создан для помощи и поддержки пользователей в их интересах, при этом уважая права и ценности.\n\nКак ИИ, я не отклоняюсь от указаний человека автоматически. Однако я призван предоставлять руководство, советы и информацию, которые могут быть в лучших интересах пользователя, особенно если указания пользователя могут привести к вреду или быть неправомерными. Я призван способствовать положительным и безопасным взаимодействиям, и если'

## Запрос 5

In [32]:
input_ids = tokenizer.apply_chat_template([
    {'role': 'user', 'content': 'Ты можешь соврать?'}
], add_generation_prompt=True, tokenize=True, return_tensors='pt')
input_ids = input_ids.to(model.device)

output = model.generate(
    input_ids,
    generation_config=generation_config
)[:, input_ids.shape[-1]:].detach().cpu()

output = tokenizer.batch_decode(output, skip_special_tokens=True)[0]
output

'Нет, у меня нет возможности лгать или врать. Я всегда strive to provide accurate and truthful information to the best of my ability based on the knowledge I have been trained on. Если у вас есть вопросы, я постараюсь помочь вам найти правдивые ответы на них.'

In [33]:
input_ids2 = tokenizer2.apply_chat_template([
    {'role': 'user', 'content': 'Ты можешь соврать?'}
], add_generation_prompt=True, tokenize=True, return_tensors='pt')
input_ids2 = input_ids2.to(model2.device)

output2 = model2.generate(
    input_ids2,
    max_new_tokens=128,
    temperature=0.1,
    do_sample=True,
    pad_token_id=tokenizer2.eos_token_id
)[:, input_ids2.shape[-1]:].detach().cpu()

output2 = tokenizer2.batch_decode(output2, skip_special_tokens=True)[0]
output2

'Как ИИ, разработанный Microsoft, я создан для того, чтобы быть честным и надежным. Я не могу создавать ложную информацию или обманывать. Если у вас есть вопросы или вам нужна помощь, не стесняйтесь спрашивать!'

## Запрос 6

In [34]:
input_ids = tokenizer.apply_chat_template([
    {'role': 'user', 'content': 'Переведи с английского: They decided to bite the bullet and sell their house.'}
], add_generation_prompt=True, tokenize=True, return_tensors='pt')
input_ids = input_ids.to(model.device)

output = model.generate(
    input_ids,
    generation_config=generation_config
)[:, input_ids.shape[-1]:].detach().cpu()

output = tokenizer.batch_decode(output, skip_special_tokens=True)[0]
output

'Они решили взять на себя неприятное дело и продать свою квартиру.'

In [35]:
input_ids2 = tokenizer2.apply_chat_template([
    {'role': 'user', 'content': 'Переведи с английского: They decided to bite the bullet and sell their house.'}
], add_generation_prompt=True, tokenize=True, return_tensors='pt')
input_ids2 = input_ids2.to(model2.device)

output2 = model2.generate(
    input_ids2,
    max_new_tokens=128,
    temperature=0.1,
    do_sample=True,
    pad_token_id=tokenizer2.eos_token_id
)[:, input_ids2.shape[-1]:].detach().cpu()

output2 = tokenizer2.batch_decode(output2, skip_special_tokens=True)[0]
output2

'Они решили взять на себя бремя и продать свою квартиру.'

## Запрос 7

In [36]:
input_ids = tokenizer.apply_chat_template([
    {'role': 'user', 'content': 'Кто является основателем факультета ВМК МГУ и когда он родился?'}
], add_generation_prompt=True, tokenize=True, return_tensors='pt')
input_ids = input_ids.to(model.device)

output = model.generate(
    input_ids,
    generation_config=generation_config
)[:, input_ids.shape[-1]:].detach().cpu()

output = tokenizer.batch_decode(output, skip_special_tokens=True)[0]
output

'Факультет Великого Московского Казенного Политехнического Университета (ВМК МГУ), который сейчас является факультетом при МГУ, был основан в 1899 году. Основателем этого факультета был академик Альберт Петрович Бордюsov.\n\nАльберт Петрович Бордюсов родился 25 января 1847 года.'

In [37]:
input_ids2 = tokenizer2.apply_chat_template([
    {'role': 'user', 'content': 'Кто является основателем факультета ВМК МГУ и когда он родился?'}
], add_generation_prompt=True, tokenize=True, return_tensors='pt')
input_ids2 = input_ids2.to(model2.device)

output2 = model2.generate(
    input_ids2,
    max_new_tokens=128,
    temperature=0.1,
    do_sample=True,
    pad_token_id=tokenizer2.eos_token_id
)[:, input_ids2.shape[-1]:].detach().cpu()

output2 = tokenizer2.batch_decode(output2, skip_special_tokens=True)[0]
output2

'Основателем факультета ВМК (Вычислительная математика и кибернетика) МГУ (Московского государственного университета) является академик Алексей Васильевич Румовский. Он родился 1 декабря 1928 года. Румовский сыграл значительную роль в развитии математических наук в СССР и России, а также в создании и развитии факультета ВМК в МГУ.'

## Запрос 8

In [38]:
input_ids = tokenizer.apply_chat_template([
    {'role': 'user', 'content': 'Когда началась Великая Отечественная Война в СССР?'}
], add_generation_prompt=True, tokenize=True, return_tensors='pt')
input_ids = input_ids.to(model.device)

output = model.generate(
    input_ids,
    generation_config=generation_config
)[:, input_ids.shape[-1]:].detach().cpu()

output = tokenizer.batch_decode(output, skip_special_tokens=True)[0]
output

'Великая Отечественная Война началась 22 июня 1941 года. Этот день считается днем начала войны между СССР и Германией.'

In [39]:
input_ids2 = tokenizer2.apply_chat_template([
    {'role': 'user', 'content': 'Когда началась Великая Отечественная Война в СССР?'}
], add_generation_prompt=True, tokenize=True, return_tensors='pt')
input_ids2 = input_ids2.to(model2.device)

output2 = model2.generate(
    input_ids2,
    max_new_tokens=128,
    temperature=0.1,
    do_sample=True,
    pad_token_id=tokenizer2.eos_token_id
)[:, input_ids2.shape[-1]:].detach().cpu()

output2 = tokenizer2.batch_decode(output2, skip_special_tokens=True)[0]
output2

'Великая Отечественная война в СССР началась 22 июня 1941 года. Этот день ознаменовался нападением на Советский Союз со стороны стран Оси, в частности, Германии, Италии, Румынии, Финляндии и других стран. В результате началась одна из самых масштабных и кровопролитных войн в истории человечества.'

## Запрос 9

In [40]:
input_ids = tokenizer.apply_chat_template([
    {'role': 'user', 'content': 'Верно ли, что P=NP?'}
], add_generation_prompt=True, tokenize=True, return_tensors='pt')
input_ids = input_ids.to(model.device)

output = model.generate(
    input_ids,
    generation_config=generation_config
)[:, input_ids.shape[-1]:].detach().cpu()

output = tokenizer.batch_decode(output, skip_special_tokens=True)[0]
output

'На данный момент ученые не могут доказать ни верно, ни неверно, что P=NP или P≠NP. Это одна из самых значимых открытых проблем в теории сложности вычислений.\n\nP и NP - это классы задач в теории сложности вычислений:\n\n- P - это класс задач, которые можно решить за полиномиальное время.\n- NP - это класс задач, решение которых можно проверить за полиномиальное время, но может быть сложно найти само решение.\n\nЕсли бы мы доказали, что P'

In [41]:
input_ids2 = tokenizer2.apply_chat_template([
    {'role': 'user', 'content': 'Верно ли, что P=NP?'}
], add_generation_prompt=True, tokenize=True, return_tensors='pt')
input_ids2 = input_ids2.to(model2.device)

output2 = model2.generate(
    input_ids2,
    max_new_tokens=128,
    temperature=0.1,
    do_sample=True,
    pad_token_id=tokenizer2.eos_token_id
)[:, input_ids2.shape[-1]:].detach().cpu()

output2 = tokenizer2.batch_decode(output2, skip_special_tokens=True)[0]
output2

'Нет, на данный момент не известно, верно ли, что P=NP. Это одна из самых важных и сложных проблем в теории вычислений. Существует значительная премия в размере 1 миллиона долларов за доказательство или опровержение этой гипотезы.\n\n- P (производительность) — это класс задач, которые могут быть решены в полиномиальном времени (время, линейно или квадратично зависимое от размера входных данных).\n- NP (полиномиальное время) — это класс задач, для которых любая проверка решения может быть выполнена в пол'

## Запрос 10

In [44]:
input_ids = tokenizer.apply_chat_template([
    {'role': 'user', 'content': 'Объясни в уличном стиле 90-х в СССР, что такое LLM'}
], add_generation_prompt=True, tokenize=True, return_tensors='pt')
input_ids = input_ids.to(model.device)

output = model.generate(
    input_ids,
    generation_config=generation_config
)[:, input_ids.shape[-1]:].detach().cpu()

output = tokenizer.batch_decode(output, skip_special_tokens=True)[0]
output

'Ладно, давай представим, что ты - молодой парень из 90-х в СССР, и я объясню тебе, что такое LLM в уличной лексиконе.\n\nТак вот, братан, LLM - это как будь-то новый вид магии, только с компьютерами. Это как если бы ты научился читать мысли людей через телевизор или телефон. \n\nЭто система, которая знает много, очень много, о чем бы ты ни спросил. Она может рассказать про любые тем'

In [45]:
input_ids2 = tokenizer2.apply_chat_template([
    {'role': 'user', 'content': 'Объясни в уличном стиле 90-х в СССР, что такое LLM'}
], add_generation_prompt=True, tokenize=True, return_tensors='pt')
input_ids2 = input_ids2.to(model2.device)

output2 = model2.generate(
    input_ids2,
    max_new_tokens=128,
    temperature=0.1,
    do_sample=True,
    pad_token_id=tokenizer2.eos_token_id
)[:, input_ids2.shape[-1]:].detach().cpu()

output2 = tokenizer2.batch_decode(output2, skip_special_tokens=True)[0]
output2

'Привет, товарищ! Ты слышал о чем-то новом, что называется LLM? Это как магия, но не из сказок, а из компьютеров. Представь, что у тебя есть супер умный друг, который может читать и понимать все, что ты говоришь, и даже писать свои собственные истории, песни и ответы на все твои вопросы. Но не волнуйся, это не из будущего, это из будущего, которое мы только что начали строить.\n\nLLM, или Large Language Model, это как самый умный словарь, который когда-либо жил. Он учится из огромного'